In [2]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from multiprocessing import Pool

In [3]:
load_dotenv()
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
HEADERS = {
    "User-Agent": "TDS Project 1",
    "Authorization": f"token {GITHUB_TOKEN}"
}
BASE_URL = "https://api.github.com"

## Scraping all users in the in the city of `Beijing` with over `500` followers

#### `users.csv` has following information about each user in Beijing with over 500 followers, with fields:

- `login`: Their Github user ID
- `name`: Their full name
- `company`: The company they work at. Clean up company names. At least make sure:
- - They're trimmed of whitespace
- - Leading `@` symbols are stripped
- - They are converted to UPPERCASE
- `location`: The city they are in
- `email`: Their email address
- `hireable`: Whether they are open to being hired
- `bio`: A short bio about them
- `public_repos`: The number of public repositories they have
- `followers`: The number of followers they have
- `following`: The number of people they are following
- `created_at`: When they joined Github


In [82]:
def get_user_details(user, **request_params):
    url = f"{BASE_URL}/users/{user['login']}"
    response = requests.get(url, **request_params)
    if response.status_code != 200:
        print(f"Error fetching user details: {response.status_code}")
        return None

    data = response.json()
    return data

In [83]:
def get_users_in_city(city, min_followers=500,  **request_params):
    page = 1
    users = pd.DataFrame([
        "login", "name", "company", "location", "email", "hireable", "bio", "public_repos", "followers", "following", "created_at"
    ])
    users = None
    while True:

        url = f"{BASE_URL}/search/users?q=location:{city}+followers:>{min_followers}&per_page=100&page={page}"
        response = requests.get(url, **request_params)
        if response.status_code != 200:
            print(f"Error fetching users: {response.status_code}")
            break

        data = response.json()
        items = data.get("items", [])

        if users is None:
            total_pages = int(response.links["last"]["url"].split(
                "=")[-1]) if "last" in response.links else 1

        for user in items:
            user_details = get_user_details(user, **request_params)
            if user_details is None:
                continue
            if users is None:
                users = pd.DataFrame(columns=user_details.keys())

            users.loc[len(users)] = user_details.values()

        print(f"Fetched page {page}/{total_pages} - {len(items)} users")

        if "next" not in response.links:
            break

        page += 1
        time.sleep(2)

    return users

In [85]:
user_details = get_users_in_city("Beijing", headers=HEADERS)

Fetched page 1/4 - 100 users
Fetched page 2/4 - 100 users
Fetched page 3/4 - 100 users
Fetched page 4/4 - 60 users


In [90]:
user_details.to_csv("./beijing_users.csv", index=False)

In [16]:
user_details = pd.read_csv("./beijing_users.csv")

In [17]:
user_details.shape

(360, 33)

In [18]:
user_details.head()

,login,id,node_id,avatar_url,gravatar_id,url,html_url,followers_url,following_url,gists_url,...,email,hireable,bio,twitter_username,public_repos,public_gists,followers,following,created_at,updated_at
0,michaelliao,470058,MDQ6VXNlcjQ3MDA1OA==,https://avatars.githubusercontent.com/u/470058...,NaN,https://api.github.com/users/michaelliao,https://github.com/michaelliao,https://api.github.com/users/michaelliao/follo...,https://api.github.com/users/michaelliao/follo...,https://api.github.com/users/michaelliao/gists...,...,askxuefeng@gmail.com,NaN,Crypto developer.,liaoxuefeng,99,0,37235,3,2010-11-06T12:21:35Z,2024-10-11T08:36:39Z
1,daimajia,2503423,MDQ6VXNlcjI1MDM0MjM=,https://avatars.githubusercontent.com/u/250342...,NaN,https://api.github.com/users/daimajia,https://github.com/daimajia,https://api.github.com/users/daimajia/followers,https://api.github.com/users/daimajia/followin...,https://api.github.com/users/daimajia/gists{/g...,...,daimajia@gmail.com,NaN,Zhenfund VP of Investment.,daimajia,89,9,24632,271,2012-10-07T02:40:06Z,2024-10-02T20:13:18Z
2,xiaolai,152970,MDQ6VXNlcjE1Mjk3MA==,https://avatars.githubusercontent.com/u/152970...,NaN,https://api.github.com/users/xiaolai,https://github.com/xiaolai,https://api.github.com/users/xiaolai/followers,https://api.github.com/users/xiaolai/following...,https://api.github.com/users/xiaolai/gists{/gi...,...,lixiaolai@gmail.com,NaN,A lifelong student.,xiaolai,54,49,19238,37,2009-11-13T18:29:42Z,2024-10-04T02:13:28Z
3,draveness,6493255,MDQ6VXNlcjY0OTMyNTU=,https://avatars.githubusercontent.com/u/649325...,NaN,https://api.github.com/users/draveness,https://github.com/draveness,https://api.github.com/users/draveness/followers,https://api.github.com/users/draveness/followi...,https://api.github.com/users/draveness/gists{/...,...,i@draven.co,True,HFT / C++ / Go,draven907,50,27,13009,28,2014-01-24T16:22:01Z,2024-10-11T06:43:11Z
4,hongyangAndroid,10704521,MDQ6VXNlcjEwNzA0NTIx,https://avatars.githubusercontent.com/u/107045...,NaN,https://api.github.com/users/hongyangAndroid,https://github.com/hongyangAndroid,https://api.github.com/users/hongyangAndroid/f...,https://api.github.com/users/hongyangAndroid/f...,https://api.github.com/users/hongyangAndroid/g...,...,623565791@qq.com,True,学习ing,NaN,102,9,12985,35,2015-01-26T07:05:45Z,2024-07-03T06:14:21Z


In [19]:
user_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   login                360 non-null    object 
 1   id                   360 non-null    int64  
 2   node_id              360 non-null    object 
 3   avatar_url           360 non-null    object 
 4   gravatar_id          0 non-null      float64
 5   url                  360 non-null    object 
 6   html_url             360 non-null    object 
 7   followers_url        360 non-null    object 
 8   following_url        360 non-null    object 
 9   gists_url            360 non-null    object 
 10  starred_url          360 non-null    object 
 11  subscriptions_url    360 non-null    object 
 12  organizations_url    360 non-null    object 
 13  repos_url            360 non-null    object 
 14  events_url           360 non-null    object 
 15  received_events_url  360 non-null    obj

In [20]:
user_details["company"].sample(20)

276                                 NaN
191                               baidu
310                      Bytedance Inc.
125                      Bytedance, Inc
297                       Alibaba Cloud
123                                 NaN
148          Institute of Software, CAS
346                                 NaN
27                 @apache  @tetrateio 
254                      DreamSourceLab
171                                  小米
37                          studygolang
139    @apache @shardingjdbc @SphereEx 
214                         Xiaomi Corp
215                             Alibaba
60                  Tsinghua University
342                 nt authority\system
102                                  美团
331                 tsinghua university
288                           @chaitin 
Name: company, dtype: object

In [21]:
def fix_company_name(company):
    if company == "null":
        return company
    return company.strip().lstrip("@").upper()

In [22]:
cols = ["login", "name", "company", "location", "email",
        "hireable", "bio", "public_repos", "followers", "following", "created_at"]

In [25]:
user_df = user_details[cols].copy(deep=True)
user_df[["login", "name", "email", "company", "bio", "created_at"]] = user_df[[
    "login", "name", "email", "company", "bio", "created_at"]].replace({None: "null"})
user_df["hireable"] = user_df["hireable"].replace({None: False})
user_df["company"] = user_df["company"].apply(fix_company_name)
user_df.head()

/tmp/ipykernel_886921/4142162028.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_df["hireable"] = user_df["hireable"].replace({None: False})


,login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at
0,michaelliao,Crypto Michael,null,"Beijing, China",askxuefeng@gmail.com,False,Crypto developer.,99,37235,3,2010-11-06T12:21:35Z
1,daimajia,代码家,ZHENFUND BEIJING,"Beijing, China",daimajia@gmail.com,False,Zhenfund VP of Investment.,89,24632,271,2012-10-07T02:40:06Z
2,xiaolai,xiaolai,INBLOCKCHAIN,beijing,lixiaolai@gmail.com,False,A lifelong student.,54,19238,37,2009-11-13T18:29:42Z
3,draveness,Draven,SPECTRA-FUND,"Beijing, China",i@draven.co,True,HFT / C++ / Go,50,13009,28,2014-01-24T16:22:01Z
4,hongyangAndroid,张鸿洋,WANANDROID.COM,"Beijing,China",623565791@qq.com,True,学习ing,102,12985,35,2015-01-26T07:05:45Z


In [26]:
user_df.shape

(360, 11)

In [27]:
user_df.to_csv("users.csv", index=False)

#### `repositories.csv` has these users' public repositories. For each user in users.csv, fetch up to the 500 most recently pushed repositories, with fields:

- `login`: The Github user ID (login) of the owner, which, BTW, is not directly in the API response.
- `full_name`: Full name of the repository
- `created_at`: When the repository was created
- `stargazers_count`: Number of stars the repository has
- `watchers_count`: Number of watchers the repository has
- `language`: The programming language the repository is written in
- `has_projects`: Whether the repository has projects enabled
- `has_wiki`: Whether the repository has a wiki
- `license_name`: Name of the license the repository is under (This is under license.key)

In [3]:
def get_user_repos(user, **request_params):
    repos = None
    page = 1
    last_page = 5
    while page <= last_page:
        url = f"{BASE_URL}/users/{user}/repos?per_page={100}&page={page}&sort=pushed"
        response = requests.get(url, **request_params)

        if response.status_code != 200:
            print(f"Error fetching user repos: {response.status_code}")
            return repos

        data = response.json()

        if 'last' in response.links:
            last_page = int(response.links['last']
                            ['url'].split('=')[-3].split('&')[0])

        if last_page > 5:
            last_page = 5

        if repos is None:
            repos = pd.DataFrame(columns=data[0].keys())

        for repo in data:
            repos.loc[len(repos)] = repo.values()

        page += 1
        time.sleep(2)

    return repos

In [4]:
def fetch_repos(users, request_params):
    repos = None
    print(
        f"Fetching repos for {len(users)} users - [{users.index[0]} - {users.index[-1]}]")
    for user in users["login"]:
        user_repos = get_user_repos(user, **request_params)
        if user_repos is None:
            continue
        if repos is None:
            repos = pd.DataFrame(columns=user_repos.columns)

        repos = pd.concat([repos, user_repos], ignore_index=True)

    print(
        f"Finished fetching Repos for {len(users)} users - [{users.index[0]} - {users.index[-1]}]")

    return repos

In [ ]:
repos = fetch_repos(user_df.iloc[:10], HEADERS)
repos.shape

In [143]:
repos.to_csv("repos.csv", index=False)

Having `360` users will take a lotttt of time to fetch their 500 repos, so let's do it in multiprocessing.

In [17]:
def fetch_repos_multi(users, processes=10, **request_params):
    user_batches = [users.iloc[i::processes] for i in range(processes)]
    with Pool(processes=processes) as pool:
        results = pool.starmap(
            fetch_repos, [(batch, request_params) for batch in user_batches])

    repos = pd.concat(results, ignore_index=True)
    return repos

In [101]:
user_df.shape

(360, 10)

In [20]:
repos = fetch_repos_multi(user_df, processes=10, headers=HEADERS)

Fetching repos for 36 users - [0 - 350]Fetching repos for 36 users - [1 - 351]Fetching repos for 36 users - [2 - 352]Fetching repos for 36 users - [3 - 353]
Fetching repos for 36 users - [4 - 354]

Fetching repos for 36 users - [6 - 356]
Fetching repos for 36 users - [7 - 357]Fetching repos for 36 users - [8 - 358]Fetching repos for 36 users - [5 - 355]




Fetching repos for 36 users - [9 - 359]
Finished fetching Repos for 36 users - [4 - 354]
Finished fetching Repos for 36 users - [9 - 359]
Finished fetching Repos for 36 users - [2 - 352]
Finished fetching Repos for 36 users - [8 - 358]
Finished fetching Repos for 36 users - [3 - 353]
Finished fetching Repos for 36 users - [6 - 356]
Finished fetching Repos for 36 users - [1 - 351]
Finished fetching Repos for 36 users - [5 - 355]
Finished fetching Repos for 36 users - [7 - 357]
Finished fetching Repos for 36 users - [0 - 350]


Let's verify the total number of repos

In [103]:
repos.shape

(29561, 81)

In [104]:
user_details["public_repos"].apply(lambda x: 500 if x > 500 else x).sum()

np.int64(29556)

So, we have some ambiguity, we are having $5$ extra repos, let's fix that.

> turns out, that ambiguity was there bcz 5 users created a new repo after we fetched their repos, so we are good to go.

In [25]:
repos.to_parquet("./repos.parquet", index=False)

##### filtering data

In [4]:
repos = pd.read_parquet("./repos.parquet")

In [5]:
cols = ["login", "full_name", "created_at", "stargazers_count",
        "watchers_count", "language", "has_projects", "has_wiki", "license_name"]

[col for col in cols if col not in repos.columns]

['login', 'license_name']

In [7]:
repos["login"] = repos["owner"].apply(lambda x: x["login"])

In [8]:
repos["license_name"] = repos["license"].apply(
    lambda x: x["key"] if x is not None else "null")

In [9]:
repo_data = repos[cols].copy(deep=True)
repo_data.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,michaelliao,michaelliao/liaoxuefeng.com,2023-11-02T11:19:06Z,35,35,JavaScript,False,False,other
1,michaelliao,michaelliao/warpdb,2016-03-26T00:32:51Z,89,89,Java,True,True,gpl-3.0
2,michaelliao,michaelliao/gitignore-online-generator,2022-11-20T02:28:38Z,54,54,HTML,False,False,cc0-1.0
3,michaelliao,michaelliao/learngit,2018-05-18T13:34:21Z,435,435,None,True,True,null
4,michaelliao,michaelliao/gitsite-cli,2023-11-05T06:54:49Z,4,4,HTML,True,True,mit


In [11]:
repo_data[["language", "has_projects", "has_wiki"]] = repo_data[["language", "has_projects", "has_wiki"]].replace(
    {False: "false", True: "true", None: "null"})

In [12]:
repo_data.head()

,login,full_name,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name
0,michaelliao,michaelliao/liaoxuefeng.com,2023-11-02T11:19:06Z,35,35,JavaScript,false,false,other
1,michaelliao,michaelliao/warpdb,2016-03-26T00:32:51Z,89,89,Java,true,true,gpl-3.0
2,michaelliao,michaelliao/gitignore-online-generator,2022-11-20T02:28:38Z,54,54,HTML,false,false,cc0-1.0
3,michaelliao,michaelliao/learngit,2018-05-18T13:34:21Z,435,435,null,true,true,null
4,michaelliao,michaelliao/gitsite-cli,2023-11-05T06:54:49Z,4,4,HTML,true,true,mit


In [13]:
repo_data.to_csv("repositories.csv", index=False)